In [1]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# tested in transformers==4.18.0, pytorch==1.7.1 
import torch
import transformers
torch.__version__, transformers.__version__

('2.0.1+cu117', '4.29.2')

*Note: the following code is for demonstration purpose. Please use GPU for fast inference on large scale dataset.*

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_initialized())
print(torch.cuda.init())

True
NVIDIA GeForce RTX 3050 Laptop GPU
True
None


### load dataset

In [4]:
df = pd.read_csv('duplicates_removed.csv') ## use your own customized dataset
df.head()

,sentence,label
0,The GeoSolutions technology will leverage Bene...,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",2
2,"For the last quarter of 2010 , Componenta 's n...",1
3,According to the Finnish-Russian Chamber of Co...,0
4,The Swedish buyout firm has sold its remaining...,0


In [5]:
df = df.dropna(subset=['sentence', 'label']) ## drop missing values

In [6]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(5319, 2) (657, 2) (591, 2)


In [8]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

### prepare dataset for fine-tuning

In [9]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/5319 [00:00<?, ? examples/s]

Map:   0%|          | 0/591 [00:00<?, ? examples/s]

Map:   0%|          | 0/657 [00:00<?, ? examples/s]

### define training options

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'finbert-sentiment-v1/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        push_to_hub=True
)

trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()   

### evaluate on testing set

In [11]:
model.eval()
trainer.predict(dataset_test).metrics

  0%|          | 0/42 [00:00<?, ?it/s]

{'test_loss': 0.4885813295841217,
 'test_accuracy': 0.786910197869102,
 'test_runtime': 8.5601,
 'test_samples_per_second': 76.751,
 'test_steps_per_second': 4.906}

### save the fine-tuned model

In [12]:
trainer.save_model('finbert-sentiment-v1/')
tokenizer.save_pretrained('finbert-sentiment-v1/')

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/419M [00:00<?, ?B/s]

Upload file runs/May29_00-49-40_ROzkurt/events.out.tfevents.1685310585.ROzkurt.3180.0:   0%|          | 1.00/6…

To https://huggingface.co/rifatozkurt/finbert-sentiment-v1
   2b33f5d..f3f6e18  main -> main

To https://huggingface.co/rifatozkurt/finbert-sentiment-v1
   f3f6e18..376adf9  main -> main



('finbert-sentiment-v1/tokenizer_config.json',
 'finbert-sentiment-v1/special_tokens_map.json',
 'finbert-sentiment-v1/vocab.txt',
 'finbert-sentiment-v1/added_tokens.json')

In [ ]:
import huggingface_hub
from huggingface_hub import notebook_login
my_token = 'my_token' #Please generate a token from huggingface
huggingface_hub.login(token = my_token)
notebook_login()

In [14]:
trainer.push_to_hub("removed duplicates from the dataset")

In [12]:
import optuna

def model_init():
    return BertForSequenceClassification.from_pretrained('rifatozkurt/finbert-sentiment-v1',num_labels=3)

In [ ]:

trainer = Trainer(
        model_init=model_init,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
)
def my_hp_space(trial: optuna.trial.Trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-5, log=True),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [1,2,3]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
    }


In [ ]:
best_run = trainer.hyperparameter_search(hp_space = my_hp_space, backend = "optuna", n_trials=4, direction="maximize")

In [18]:
best_run

BestRun(run_id='3', objective=0.7631133671742809, hyperparameters={'learning_rate': 4.450562175829955e-06, 'num_train_epochs': 2, 'per_device_train_batch_size': 4}, run_summary=None)

In [19]:
for x, y in best_run.hyperparameters.items():
    setattr(trainer.args, x, y)

In [25]:
trainer.args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=Hub

In [ ]:
trainer.train()

In [27]:
trainer.evaluate()

  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.5863251090049744,
 'eval_accuracy': 0.7631133671742809,
 'eval_runtime': 6.7693,
 'eval_samples_per_second': 87.306,
 'eval_steps_per_second': 5.466,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()